# FC calculation script
The input file is in `.h5` format. This one is used to generate some statistical properties of the functional connectivity data.

In [ ]:
# import h5py
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pandas as pd

In [ ]:
# filename = "/mnt/c/Users/wayne/Downloads/test.h5"
# labels = ['aCNG-L', 'aCNG-R','mCNG-L','mCNG-R','pCNG-L','pCNG-R', 'HIP-L','HIP-R','PHG-L','PHG-R','AMY-L','AMY-R', 'sTEMp-L','sTEMP-R','mTEMp-L','mTEMp-R']
# with h5py.File(filename, "r") as f:
#     a_group_key = list(f.keys())[0]

#     # Get the data
#     data = np.array(list(f[a_group_key]))


# real_data = pd.DataFrame(data[:,:,0,0], columns = labels, index = labels)
# mask = np.triu(real_data)
# sns.heatmap(real_data, mask = mask)

In [ ]:
# contain = np.empty(120, dtype=float)
# ind = np.empty(120, dtype=object)
# pair_regions = np.tril(real_data, k=-1)
# pr = pd.DataFrame(pair_regions, columns=labels, index=labels)
# z = 0
# for i in range(1,16,1):
#     for j in range(i):
#         if pr.iloc[i,j] != 0:
#             contain[z] =  pr.iloc[i,j]
#             ind[z] = labels[i] + "&" + labels[j]
#             z += 1
# pv = pd.DataFrame(data = contain.T, index = ind.T, columns = ["FC_value"])
# # pv["Rank"] = pv["FC_value"].rank()
# pv.sort_values("FC_value", inplace=True)
# plt.figure(figsize=(10,10))
# sns.heatmap(pv)

In [ ]:
import scipy.io
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import io
import numpy as np
mat = scipy.io.loadmat('/home/yat-lok/workspace/data4project/lateralization/ts_fmri/AD-TS/0306A/ROISignals_0306A.mat')
all = mat['ROISignals']

# filename = '/mnt/c/Users/wayne/Desktop/Functional TS/Region_Labels_90ROIs.txt'
# list_ = open(filename).read().split()
# region = list_[1::2]

# fc = np.ones((88,88))
# for i in range(88):
#     for j in range(88):
#         fc[i,j] = np.corrcoef(all[:,i], all[:,j])

df = pd.DataFrame(all, columns = range(16))
fc = df.corr(method='pearson')
print(fc)

fig = plt.figure(figsize=(15,12),dpi = 300)
axes = fig.add_subplot(111)
#sns.heatmap(pd.DataFrame(fc), cmap = "coolwarm", annot=False, ax = axes)
# sns.heatmap(pd.DataFrame(fc), cmap = "coolwarm", annot=False, ax = axes, xticklabels=region[:89], yticklabels=region[:89])


In [ ]:
# stat
# preprocessing
FC_tria = np.tril(fc, k=0)
ff = FC_tria.flatten(order='A')
fc_ff = [f for f in ff if f < 1 and f >0]

# mean
mean_fc = np.mean(fc_ff)
print(f"The mean of the FC is {mean_fc}")

# histogram
fig = plt.figure(figsize=(5,5),dpi = 300)
axes = fig.add_subplot(111)
axes.set_title("The Histogram of the FC distribution")
sns.histplot(fc_ff, ax=axes)
plt.show()

# pairs
fc.columns = region[:88]
fc.index = region[:88]

# for the max
max_fc = pd.DataFrame(columns=["from", "to", "value"])
for i in region[:88]:
    max_v = fc[i].nlargest(2)
    print(f"The positive highest correlation value between {max_v.index[0]} and {max_v.index[1]} is {max_v[1]}")
    _fc = pd.DataFrame.from_dict([{"from": max_v.index[0], "to": max_v.index[1], "value": max_v[1]}])
    max_fc = pd.concat([max_fc, _fc], ignore_index=True)
max_fc.to_excel("path_in_your_computer/max_fc.xlsx") ########### plz change the path

# for the lowest
min_fc = pd.DataFrame(columns=["from", "to", "value"])
for i in region[:88]:
    min_v = fc[i].idxmin()
    print(f"The negative highest anticorrelation value between {i} and {min_v} is {fc.loc[min_v, i]}")
    _fc = pd.DataFrame.from_dict([{"from": i, "to": min_v, "value": fc.loc[min_v, i]}])
    min_fc = pd.concat([min_fc, _fc], ignore_index=True)
min_fc.to_excel("path_in_your_computer/min_fc.xlsx") ########### plz change the path



In [ ]:
### dFC
import scipy.io
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import io
import numpy as np
import sys
sys.path.append('/home/yat-lok/workspace/Network-science-Toolbox/Python')
from TS2dFCstream import TS2dFCstream
from dFCstream2Trimers import dFCstream2Trimers
from dFCstream2MC import dFCstream2MC

mat = scipy.io.loadmat('/home/yat-lok/workspace/data4project/lateralization/ts_fmri/AD-TS/0306A/ROISignals_0306A.mat')
all = mat['ROISignals']

# filename = '/mnt/c/Users/wayne/Desktop/Functional TS/Region_Labels_90ROIs.txt'
# list_ = open(filename).read().split()
# region = list_[1::2]

# fc = np.ones((88,88))
# for i in range(88):
#     for j in range(88):
#         fc[i,j] = np.corrcoef(all[:,i], all[:,j])

df = pd.DataFrame(all, columns = range(16))
dFCstream = TS2dFCstream(df.to_numpy(), 10, None, '3D')

# fig = plt.figure(figsize=(20,100))
# #sns.heatmap(pd.DataFrame(fc), cmap = "coolwarm", annot=False, ax = axes)
# for i in range(dFCstream.shape[2]):
#     axes = fig.add_subplot(10,2,i+1)
#     axes.set_title(f"From Time {i*10} to {(i+1)*10}")
#     sns.heatmap(dFCstream[:,:,i], cmap = "coolwarm", annot=False, ax = axes)
#     #sns.heatmap(dFCstream[:,:,i], cmap = "coolwarm", annot=False, ax = axes, xticklabels=region[:89], yticklabels=region[:89])
# plt.show()
print(dFCstream.shape)

delve_time = []
for i in range(dFCstream.shape[2]-1):
    flat1 = np.triu(dFCstream[:,:,i]).flatten()[np.nonzero(np.triu(dFCstream[:,:,i]).flatten())]
    flat2 = np.triu(dFCstream[:,:,i+1]).flatten()[np.nonzero(np.triu(dFCstream[:,:,i+1]).flatten())]
    delve = 1 - np.corrcoef(flat1, flat2)[0,1]
    delve_time.append(delve)
delve_time.append(0)
#fig = plt.figure(figsize=(100,dFCstream.shape[2]))
fig = plt.figure(figsize=(100,100))
#sns.heatmap(pd.DataFrame(fc), cmap = "coolwarm", annot=False, ax = axes)
for i in range(dFCstream.shape[2]):
    #axes = fig.add_subplot(2,10,i+1)
    axes = fig.add_subplot(111)
    axes.set_title(f"From Time {i*10} to {(i+1)*10}")
    print(f"The dFC speed between {i*10} sec and {(i+1)*10} sec is {delve_time[i]}")
    sns.heatmap(dFCstream[:,:,i], cmap = "coolwarm", annot=False, ax = axes, cbar=[])
    plt.savefig(f'{i}.png')
    #sns.heatmap(dFCstream[:,:,i], cmap = "coolwarm", annot=False, ax = axes, xticklabels=region[:89], yticklabels=region[:89])
plt.show()